In [177]:
import warnings
warnings.filterwarnings("ignore")

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

# statistics
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# prophet by Facebook
from fbprophet import Prophet

# machine learning: XGB
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper

In [178]:
saat =pd.read_excel('C:/Users/bran/Desktop/Hist/saat-xgb1.xlsx','Hoja1')
saat.to_csv('saatxgb1.csv', encoding='utf-8',index_label=None)


In [179]:
saat = pd.read_csv('C:/Users/bran/Desktop/Hist/saatxgb1.csv')
saat.tail()

,Unnamed: 0,Mes,Dia_semana,Hora,Casos
18283,18283,1,3,19,4
18284,18284,1,3,20,11
18285,18285,1,3,21,2
18286,18286,1,3,22,1
18287,18287,1,3,23,0


In [180]:
saat.isnull().sum()

Unnamed: 0    0
Mes           0
Dia_semana    0
Hora          0
Casos         0
dtype: int64

In [181]:



predictors = [x for x in saat.columns if x not in ['Casos']]
y = np.log(saat.Casos) # log transformation of Sales
X = saat

# split the data into train/test set
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
X.head()

,Unnamed: 0,Mes,Dia_semana,Hora,Casos
0,0,1,5,0,2
1,1,1,5,1,1
2,2,1,5,2,1
3,3,1,5,3,2
4,4,1,5,4,1


In [182]:
# predictors
X.columns

Index(['Unnamed: 0', 'Mes', 'Dia_semana', 'Hora', 'Casos'], dtype='object')

In [183]:


# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle
#from keras import backend as K
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))
#    return np.abs(K.sqrt(K.mean(K.square(yhat - y), axis=-1)))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

In [184]:
# base parameters
params = {
    'booster': 'gbtree', 
    'objective': 'reg:linear', # regression task
    'subsample': 0.70, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 0.85, # 85% of features used
    'eta': 0.05, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results

In [185]:
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train[predictors], y_train)
dtest = xgb.DMatrix(X_test[predictors], y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 100, evals = watchlist,
                      early_stopping_rounds = 300, feval = rmspe_xg, verbose_eval = True)

[0]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 300 rounds.
[1]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[2]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[3]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[4]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[5]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[6]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[7]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[8]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[9]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[10]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[11]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[12]	train-rmse:nan	test-rmse:nan	train-rmspe:nan	test-rmspe:nan
[13]	train-rmse:nan

In [151]:
params_sk = {'max_depth': 10, 
            'n_estimators': 300, # the same as num_rounds in xgboost
            'objective': 'reg:linear', 
            'subsample': 0.8, 
            'colsample_bytree': 0.85, 
            'learning_rate': 0.1, 
            'seed': 42}     

skrg = XGBRegressor(**params_sk)

skrg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=0.8)

In [152]:
import scipy.stats as st

params_grid = {  
    'learning_rate': st.uniform(0.01, 0.3),
    'max_depth': list(range(10, 20, 2)),
    'gamma': st.uniform(0, 10),
    'reg_alpha': st.expon(0, 50)}

search_sk = RandomizedSearchCV(skrg, params_grid, cv = 5) # 5 fold cross validation
search_sk.fit(X_train, y_train)

# best parameters
print(search_sk.best_params_); print(search_sk.best_score_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').